# Image Classification

In case you want to try the original C# solution, you can find it in <a href="https://github.com/dotnet/machinelearning-samples/tree/master/samples/csharp/getting-started/DeepLearning_ImageClassification_Training">this repository</a>.

## Make a prediction

Assuming we have done the previous process of training, we're now going to consume the model in order to obtain predictions.

## Step 1 - Import NuGet packages

Necessary NuGet packages can easily be imported to use it in a Jupyter Notebook using the following code. 
In this case we'll also load some dll to import some auiliary functions.

In [ ]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML,1.4"
#r "nuget:Microsoft.ML.Vision"

#r "nuget:SciSharp.Tensorflow.Redist"  

In [ ]:
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

We will need to define the output shape class for our predictions and for our image input data 

In [ ]:
public class ImagePrediction
{
    [ColumnName("Score")]
    public float[] Score;

    [ColumnName("PredictedLabel")]
    public string PredictedLabel;
}

public class InMemoryImageData
{
    public InMemoryImageData(byte[] image, string label, string imageFileName)
    {
        Image = image;
        Label = label;
        ImageFileName = imageFileName;
    }

    public readonly byte[] Image;

    public readonly string Label;

    public readonly string ImageFileName;
}

We also define a helper class to load images from a Directory

In [ ]:
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Reflection;

public class FileUtils
{
    public static IEnumerable<(string imagePath, string label)> LoadImagesFromDirectory(
        string folder,
        bool useFolderNameasLabel)
    {
        var imagesPath = Directory
            .GetFiles(folder, "*", searchOption: SearchOption.AllDirectories)
            .Where(x => Path.GetExtension(x) == ".jpg" || Path.GetExtension(x) == ".png");

        return useFolderNameasLabel
            ? imagesPath.Select(imagePath => (imagePath, Directory.GetParent(imagePath).Name))
            : imagesPath.Select(imagePath =>
            {
                var label = Path.GetFileName(imagePath);
                for (var index = 0; index < label.Length; index++)
                {
                    if (!char.IsLetter(label[index]))
                    {
                        label = label.Substring(0, index);
                        break;
                    }
                }
                return (imagePath, label);
            });
    }

    public static IEnumerable<InMemoryImageData> LoadInMemoryImagesFromDirectory(
        string folder,
        bool useFolderNameAsLabel = true)
        => LoadImagesFromDirectory(folder, useFolderNameAsLabel)
            .Select(x => new InMemoryImageData(
                image: File.ReadAllBytes(x.imagePath),
                label: x.label,
                imageFileName: Path.GetFileName(x.imagePath)));

    public static string GetAbsolutePath(Assembly assembly, string relativePath)
    {
        var assemblyFolderPath = new FileInfo(assembly.Location).Directory.FullName;

        return Path.Combine(assemblyFolderPath, relativePath);
    }
}

As berfore, we'll also store some paths for a more confortable handling of them

In [ ]:
var imagesFolderPathForPredictions = Path.Combine("Data", "assets", "inputs", "images-for-predictions");
var imageClassifierModelZipFilePath = Path.Combine("Data", "assets", "outputs", "imageClassifer.zip");

## Step 2 - Scoring

This process consist on loading your custom trained ML.NET model and performing a few sample predictions the same way a hypothetical end-user app could do.

In regards the code, you first need to load the model created during model training app execution.

In [ ]:
var mlContext = new MLContext(seed: 1);

Console.WriteLine($"Loading model from: {imageClassifierModelZipFilePath}");

// Load the model
var loadedModel = mlContext.Model.Load(imageClassifierModelZipFilePath, out var modelInputSchema);

Then, your create a predictor engine object and finally make a few sample predictions by using the first image of the folder assets/inputs/images-for-predictions which has just a few images that were not used when training the model.

Note that now, when scoring, you only need the InMemoryImageData type which has the in-memory image.

That image could also be coming thorugh any other channel instead of loading it from a file. For instance, the ImageClassification.WebApp in this same solution gets the image to use for the prediction through HTTP as an image provided by an end-user.

In [ ]:
// Create prediction engine to try a single prediction (input = ImageData, output = ImagePrediction)
var predictionEngine = mlContext.Model.CreatePredictionEngine<InMemoryImageData, ImagePrediction>(loadedModel);

//Predict the first image in the folder
var imagesToPredict = FileUtils.LoadInMemoryImagesFromDirectory(imagesFolderPathForPredictions, false);

var imageToPredict = imagesToPredict.First();

The prediction engine receives as parameter an object of type InMemoryImageData (containing 2 properties: Image and ImageFileName). The ImageFileName is not used byt the model. You simple have it there so you can print the filename out out when showing the prediction. The prediction is only using the image's bits in the byte[] Image field.

Then the model returns and object of type ImagePrediction, which holds the PredictedLabel and all the Scores for all the classes/types of images.

Since the PredictedLabel is already a string it'll be shown in the console. About the score for the predicted label, we just need to take the highest score which is the probability for the predicted label.

In [ ]:
// Measure #1 prediction execution time.
var watch = System.Diagnostics.Stopwatch.StartNew();

var prediction = predictionEngine.Predict(imageToPredict);

// Stop measuring time.
watch.Stop();
var elapsedMs = watch.ElapsedMilliseconds;
Console.WriteLine("First Prediction took: " + elapsedMs + "mlSecs");

// Measure #2 prediction execution time.
var watch2 = System.Diagnostics.Stopwatch.StartNew();

var prediction2 = predictionEngine.Predict(imageToPredict);

// Stop measuring time.
watch2.Stop();
var elapsedMs2 = watch2.ElapsedMilliseconds;
Console.WriteLine("Second Prediction took: " + elapsedMs2 + "mlSecs");

// Get the highest score and its index
var maxScore = prediction.Score.Max();

////////
// Double-check using the index
var maxIndex = prediction.Score.ToList().IndexOf(maxScore);
VBuffer<ReadOnlyMemory<char>> keys = default;
predictionEngine.OutputSchema[3].GetKeyValues(ref keys);
var keysArray = keys.DenseValues().ToArray();
var predictedLabelString = keysArray[maxIndex];
////////

Console.WriteLine($"Image Filename : [{imageToPredict.ImageFileName}], " +
                  $"Predicted Label : [{prediction.PredictedLabel}], " +
                  $"Probability : [{maxScore}] "
                  );

//Predict all images in the folder
//
Console.WriteLine("");
Console.WriteLine("Predicting several images...");

foreach (var currentImageToPredict in imagesToPredict)
{
    var currentPrediction = predictionEngine.Predict(currentImageToPredict);

    Console.WriteLine(
        $"Image Filename : [{currentImageToPredict.ImageFileName}], " +
        $"Predicted Label : [{currentPrediction.PredictedLabel}], " +
        $"Probability : [{currentPrediction.Score.Max()}]");
}

# TensorFlow DNN Transfer Learning background information

This sample app is retraining a TensorFlow model for image classification. As a user, you could think it is pretty similar to this other sample <a href="https://github.com/dotnet/machinelearning-samples/tree/master/samples/csharp/getting-started/DeepLearning_TensorFlowEstimator"> Image classifier using the TensorFlow Estimator featurizer</a>. However, the internal implementation is very different under the covers. In that mentioned sample, it is using a 'model composition approach' where an initial TensorFlow model (i.e. InceptionV3 or ResNet) is only used to featurize the images and produce the binary information per image to be used by another ML.NET classifier trainer added on top (such as LbfgsMaximumEntropy). Therefore, even when that sample is using a TensorFlow model, you are training only with a ML.NET trainer, you don't retrain a new TensorFlow model but train an ML.NET model. That's why the output of that sample is only an ML.NET model (.zip file).

In contrast, this sample is natively retraining a new TensorFlow model based on a Transfer Learning approach but training a new TensorFlow model derived from the specified pre-trained model (Inception V3 or ResNet).

The important difference is that this approach is internally retraining with TensorFlow APIs and creating a new TensorFlow model (.pb). Then, the ML.NET .zip file model you use is just like a wrapper around the new retrained TensorFlow model. This is why you can also see a new .pb file generated after training:

<img src="https://user-images.githubusercontent.com/1712635/64131693-26fa7680-cd7f-11e9-8010-89c60b71fe11.png">

In the screenshot below you can see how you can see that retrained TensorFlow model (custom_retrained_model_based_on_InceptionV3.meta.pb) in <b>Netron</b>, since it is a native TensorFlow model:

<img src="https://user-images.githubusercontent.com/1712635/64131904-9d4ba880-cd80-11e9-96a3-c2f936f8c5e0.png">

<b>Benefits</b>:
<ul>
<li><b>Train and inference using GPU:</b> When using this native DNN approach based on TensorFlow you can either use the CPU or GPU (if available) for a better performance (less time needed for training and scoring).
<li><b>Reuse across multiple frameworks and platforms:</b> This ultimately means that since you natively trained a Tensorflow model, in addition to being able to run/consume that model with the ML.NET 'wrapper' model (.zip file), you could also take the .pb TensorFlow frozen model and run it on any other framework such as Python/Keras/TensorFlow, or a Java/Android app or any framework that supports TensorFlow.
<li><b>Flexibility and performace:</b> Since ML.NET is internally retraining natively on Tensorflow layers, the ML.NET team will be able to optimize further and take multiple approaches like training on the last layer or training on multiple layers across the TensorFlow model and achive better quality levels.
</ul>